<a href="https://colab.research.google.com/github/Dyfox100/CUDA-Tutorials/blob/main/Matrix_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Matrix / Vector Operations in CUDA

### But first, set up the environment

In [ ]:
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!git clone https://github.com/Dyfox100/CUDA-Tutorials

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243
  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-okgze4q5
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-okgze4q5
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=6797992bd70e62089135bc7c664ac3f100e8f9ed33b5c38b2334c13720e4c469
  Stored in directory: /tmp/pip-ephem-wheel-cache-dta08eds/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
Cloning into 'CUDA-Tutorials'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 40 (delta 20), reused 0 (delta 0), pack-reused 0
Unp

### Adding Two Vectors

In this next piece of code, we'll add two vectors. 

In general, we'd like each thread to only work on one piece of data, but in this example we're going to write a loop. 

Adding a loop ensures we can process all of the elements if the number of data points exceeds the number of threads that we can run on the device.

This uses a 1 dimensional grid, 1 dimensional blocks, and a grid stride loop. 

Grid stride means that each thread operates on one element then adds the total number of threads in the grid to get the index of the next element. 

We'll use a multiple of 32 for the block size to avoid wasting time at the end of each block. 


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

__global__ void add(int size, float *x, float *y) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;

    for (int i = index; i < size; i += stride) {
        y[i] = x[i] + y[i];
    }
}


int main() {
    int size = 1000000000;
    float *x, *y;

    // Allocate space for both the vectors on both the host and device.
    cudaMallocManaged(&x, size*sizeof(float));
    cudaMallocManaged(&y, size*sizeof(float));

    for (int i = 0; i < size; i++) {
        x[i] = 2.0f;
        y[i] = -1.0f;
    }

    // Launch kernel with 16 blocks with 512 threads in each block.
    // 512 / 32 == 16, so the block size is a multiple of 32. 
    // This is 8192 threads, so the loop is necessary.
    add<<<16, 512>>>(size, x, y);

    cudaDeviceSynchronize();

    for(int i = 0; i < size; i++) {
        if (abs(y[i] - 1.0f) > 0.001f) {
            printf("Error is greater than 0.001! Value is: %f", y[i]);
        }
    }

    // Now we can start a very large number of threads so the loop isn't really 
    // necessary. Note 1024 is the maximum number of threads in a block.
    add<<<1048576, 1024>>>(size, x, y);

    cudaDeviceSynchronize();

    for(int i = 0; i < size; i++) {
        if (abs(y[i] - 3.0f) > 0.001f) {
            printf("Error is greater than 0.001! Value is: %f", y[i]);
        }
    }
    printf("Done! No errors detected!\n");
    printf("First value in y is: %f and should be 3.0\n", y[0]);
    printf("Wow that was quick. We just added a billion floating point numbers twice!\n");
    cudaFree(x);
    cudaFree(y);
    return 0;
}


Done! No errors detected!
First value in y is: 3.000000 and should be 3.0
Wow that was quick. We just added a billion floating point numbers twice!



### Dot Product
To calculate the dot product we need to multiply each pair of elements in two vectors, and sum them.

To do this, we'll use shared memory on a thread block to calculate a local sum,
then reduce over all thread blocks.

This will require a few new things. First off we'll need to use shared memory, secondly, we'll need to synchronize threads, and third, we'll need to use atomic operations. 

Each thread block has a piece of fast shared memory associated with it. This shared memory segment is only accessible from inside each thread block. 

To calculate the dot product on on thread block's id's, we'll create a shared memory segment to hold all of the individual products. This can be done using the \__shared__ key word.

Next we'll reduce / sum the values in each thread block's array of products. But first we'll need to synchronize the thread block. If each thread in the thread block isn't done adding their product to the shared array, we will get weird results.

Waiting for all threads to get to a certain point inside a thread block can be accomplished by using the \__syncthreads function.

Next we'll actually do the reduction to get each thread blocks portion of the dot product. But we now need to add all of the thread blocks portions together. We cna just add them to the result variable, but we need to be careful. Thread blocks run concurrently, so imagine what happens if thread block 4 reads the result variable, then thread block 5 writes to the thread block variable, then thread block 4 does it's add and writes to the result variable. The result would completly miss the contribution from block 5 becuase it wasn't there when block 4 read the result variable!

To fix this we can use an atomic operation. Atomic operations can read a memory location, perform some operation, then write back to the memory location without having anyone else read / write to that memory location. We'll use an atomicAdd here.


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define SIZE 1000000000
#define THREADS_PER_BLOCK 1024

__global__ void dot(float *x, float *y, float *result) {
    __shared__ float blockProducts[THREADS_PER_BLOCK];

    int index = blockIdx.x * blockDim.x + threadIdx.x;
    blockProducts[threadIdx.x] = x[index] * y[index];

    __syncthreads();

    
    if (threadIdx.x == 0) {
        float blockSum = 0.0;
        for (int i = 0; i < THREADS_PER_BLOCK; i++) {
            blockSum += blockProducts[i];
        }
        atomicAdd(result, blockSum);
    }
}


int main() {
    float *x, *y, *result;

    // Allocate space for both the vectors on both the host and device.
    cudaMallocManaged(&x, SIZE*sizeof(float));
    cudaMallocManaged(&y, SIZE*sizeof(float));

    // Allocate space for the result.
    cudaMallocManaged(&result, sizeof(float));

    for (int i = 0; i < SIZE; i++) {
        x[i] = 1.0f;
        y[i] = -1.0f;
    }

    int blocks = (SIZE + (THREADS_PER_BLOCK - 1)) / THREADS_PER_BLOCK;
    dot <<< blocks, THREADS_PER_BLOCK >>>(x, y, result);
    cudaDeviceSynchronize();

    if (abs(*result + SIZE) > 1.0) {
        printf("Error in dot product, dot product is: %f", *result);
    }
    else {
      printf("Done! No errors detected! Dot product is: %f and should be -%d\n", *result, SIZE);  
    }
    
    cudaFree(x);
    cudaFree(y);
    cudaFree(result);
    return 0;
}


Done! No errors detected! Dot product is: -1000000000.000000 and should be -1000000000



### Vector Matrix Multiplication

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>

#define COLS 10000
#define ROWS 20000


__global__ void matVecProd(float *mat, float *vec, float *result, int rows, int cols) {
    int tid=threadIdx.x+blockIdx.x*blockDim.x;
    float sum=0;
    if(tid < cols){
      for(int i = 0; i < rows; i++)
        sum += vec[i] * mat[(i*cols) + tid];
      result[tid] = sum;
    }
}


int main() {
    float *mat, *vec, *result;

    int cols = COLS;
    int rows = ROWS;

    cudaMallocManaged(&mat, sizeof(float) * cols * rows);
    cudaMallocManaged(&vec, sizeof(float) * rows);
    cudaMallocManaged(&result, sizeof(float) * cols);

     for (int i = 0; i < rows; i++) {
        vec[i] = 1.0f;
    }

    for (int i = 0; i < rows; i++) {
      for (int j = 0; j < cols; j++) {
        mat[i * cols + j] = 1.0f;
      }
    }
    int threadsBlock = 32 * 5;
    int blocks = (COLS * 2) / threadsBlock + 1;
    matVecProd<<<blocks, threadsBlock >>>(mat, vec, result, rows, cols);
    cudaDeviceSynchronize();

    for (int i = 0; i < cols; i++) {
        if (abs(result[i] - 20000) > 1) {
            printf("Error! Value at index %d in result vector is %f\n", i, result[i]);
        }
    }

    printf("No errors, all elements in result array are %f", result[0]);
    return 0;

}



No errors, all elements in result array are 20000.000000


### Matrix Matrix Multiplication

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>



__global__ void matMatProd(float *mat1, float *mat2, float *result, int rows, int cols) {
    
}


int main() {
    float *mat1, *mat2, *result;

    int m, n;
    m = 10000;
    n = 20000;


    cudaMallocManaged(&mat1, sizeof(float) * m * n);
    cudaMallocManaged(&mat2, sizeof(float) * n * m);
    cudaMallocManaged(&result, sizeof(float) * m * m);

    for (int i = 0; i < m; i++) {
        for (int j = 0; j < n; j++) {
            mat1[i * m + j] = 1.0f;
        }
    }

    for (int i = 0; i < n; i++) {
        for (int j = 0; j < m; j++) {
            mat2[i * n + j] = 1.0f;
        }
    }




    printf("No errors, all elements in result array are %f", result[0]);

    cudaFree(mat1);
    cudaFree(mat2);
    cudaFree(result);
    return 0;

}



No errors, all elements in result array are 20000.000000


In [ ]:
!nvcc  ./drive/MyDrive/test.cpp -I/usr/include/opencv -lopencv_core -lopencv_highgui -lopencv_imgproc -lopencv_imgcodecs -lopencv_core 


./drive/MyDrive/test.cpp:9:1: error: ‘__global__’ does not name a type; did you mean ‘__locale_t’?
 __global__ void brighten()
 ^~~~~~~~~~
 __locale_t


In [ ]:
!./a.out


Starting!
The first value in the r array is: 76
